## Multi Object Tracking sample
This notebook is an example of how to perform object detection with multi-object tracking (MOT) from a video file to count vehicle traffic. The annotated video is saved into new file with `_mot_annotated` suffix.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you need to specify the appropriate `hw_location` option.

You also need to specify your cloud API access token in [env.ini](../../env.ini) file, located in the same repository as this notebook.

In [ ]:
# make sure degirum-tools package and other dependencies are installed
!pip show degirum-tools || pip install degirum-tools

#### Specify where you want to run your inferences, model zoo url, model name and image source

In [26]:
# hw_location: where you want to run inference
#     @cloud to use DeGirum cloud
#     @local to run on local machine
#     IP address for AI server inference
# model_zoo_url: url/path for model zoo
#     cloud_zoo_url: valid for @cloud, @local, and ai server inference options
#     '': ai server serving models from local folder
#     path to json file: single model zoo in case of @local inference
# model_name: name of the model for running AI inference
# input_filename: path to the video file to annotate
hw_location = "@cloud"
model_zoo_url = "https://cs.degirum.com/degirum/public"
model_name = "yolo_v5s_person_det--512x512_quant_n2x_orca1_1"
input_filename = "../../images/store.mp4"

#### The rest of the cells below should run without any modifications

In [27]:
import degirum as dg, degirum_tools
from pathlib import Path
import IPython.display

# configure for Google Colab
degirum_tools.configure_colab()

# connect to AI inference engine getting token from env.ini file
zoo = dg.connect(hw_location, model_zoo_url, degirum_tools.get_token())

# load object detection model
model = zoo.load_model(
    model_name, overlay_show_probabilities=True, overlay_line_width=1
)

# video input and output
orig_path = Path(input_filename)


# Degirum tracker

In [ ]:
import cv2

ann_path = "../../workarea/" + orig_path.stem + "_mot_annotated_dg" + orig_path.suffix
    
with degirum_tools.open_video_stream(input_filename) as stream:
    image_w = int(stream.get(cv2.CAP_PROP_FRAME_WIDTH))
    image_h = int(stream.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(stream.get(cv2.CAP_PROP_FPS))

    with degirum_tools.Display("MoT") as display, degirum_tools.open_video_writer(
        str(ann_path), image_w, image_h
    ) as writer:
        tracker = degirum_tools.ObjectTracker(
            class_list=["person"],
            track_buffer=5*fps,
            trail_depth=0,
            track_thresh = 0.35,
            match_thresh = 0.9999,
            anchor_point=degirum_tools.AnchorPoint.BOTTOM_CENTER,
        )

        progress = degirum_tools.Progress(int(stream.get(cv2.CAP_PROP_FRAME_COUNT)))
        for result in model.predict_batch(degirum_tools.video_source(stream)):
            tracker.analyze(result)
            image = tracker.annotate(result, result.image_overlay)
            
            writer.write(image)
            display.show(image)
            progress.step()

# Supervision Tracker

In [ ]:
import cv2, numpy as np
import supervision as sv
from supervision.tracker.byte_tracker.basetrack import BaseTrack

ann_path = "../../workarea/" + orig_path.stem + "_mot_annotated_sv" + orig_path.suffix

with degirum_tools.open_video_stream(input_filename) as stream:
    image_w = int(stream.get(cv2.CAP_PROP_FRAME_WIDTH))
    image_h = int(stream.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(stream.get(cv2.CAP_PROP_FPS))

    with degirum_tools.Display("MoT") as display, degirum_tools.open_video_writer(
        str(ann_path), image_w, image_h
    ) as writer:

        BaseTrack._count = 0
        byte_tracker = sv.ByteTrack(
            track_thresh = 0.35, 
            track_buffer = 5 * fps,
            match_thresh = 0.9999,
            frame_rate = fps
        )        

        label_annotator = sv.LabelAnnotator(text_position = sv.Position.TOP_LEFT)

        progress = degirum_tools.Progress(int(stream.get(cv2.CAP_PROP_FRAME_COUNT)))
        for result in model.predict_batch(degirum_tools.video_source(stream)):

            detections = sv.Detections(np.array([r["bbox"] for r in result.results]))
            detections.confidence = np.array([r["score"] for r in result.results])
            detections.class_id = np.array([r["category_id"] for r in result.results])
            detections = byte_tracker.update_with_detections(detections)

            labels = [f"#{tracker_id}" for tracker_id in detections.tracker_id ]        
        
            image = label_annotator.annotate( scene = result.image_overlay, detections = detections, labels = labels )            
            
            writer.write(image)
            display.show(image)
            progress.step()